In [1]:
import os
import pandas as pd
path = "/Users/anastasia/Documents/Python"
os.chdir(path)



In [2]:
#Данные
df = pd.read_csv('/Users/anastasia/Downloads/compressed_data.csv', sep=';')
df

,SKU,Item combination ID,Name,Category,Brand,Seller,Color,Balance,Comments,Final price,...,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89
1,149575719,126693294,Носки капроновые телесные набор 10 пар,Акции/Распродажа 11.11/Женщинам/Белье/Колготки...,NIKII,ИП Адюшкин Н Р,светло-бежевый,286,18759,136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,266510489,244669663,Кроссовки зимние,NaN,Thomas Mitchell,ИП Нестеров И А,"красный, черный",17,26,5876,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,266508481,245279769,Кроссовки зимние,NaN,Thomas Mitchell,ИП Нестеров И А,"зеленый, серый, бежевый, синий",75,24,6247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,179066374,163341676,кроссовки,NaN,Trendy Feet,Trendy Feet,бело-синий,21,98,3360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,201392334,182598418,LITE Полуботинки женские осень натуральная кож...,Акции/Распродажа 11.11/Обувь/Женская/Ботинки и...,Mascotte,ООО Радость,"желтый, горчичный, желто-оранжевый",24,487,3179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,271353738,155081746,Туфли классические кожаные броги,Акции/Распродажа 11.11/Обувь/Мужская/Ботинки и...,Mascotte,ООО Радость,"темно-коричневый, коричневый, темный горький ш...",70,62,7850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,217831686,154017289,Кроссовки на шнурках,Акции/Распродажа 11.11/Обувь/Детская/Для мальч...,PlayToday,АО Плэйтудэй СНГ,"серый, зеленый, желтый",41,839,1276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,212290794,191730754,Лоферы женские замшевые,NaN,Ausgut,ИП Ходырев А Г,изумрудно-зеленый,48,91,4905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,207442697,187808846,Кроссовки Etonic Fanes,Акции/Распродажа 11.11/Спорт/Бег/Ходьба/Бег/Об...,Etonic,ООО Спортмастер,"розовый, фуксия",56,18,3336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Яркость, Контрастность, Человек, Энтропия

In [3]:
#Обработка на граф фичи базовые
import requests
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import warnings
from urllib3.exceptions import InsecureRequestWarning
from urllib.request import urlopen
import matplotlib.pyplot as plt
from ultralytics import YOLO
from scipy.fftpack import fft2, ifft2

# Отключение предупреждений InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)

def analyze_image(url):
    response = requests.get(url, verify=False)
    img_array = np.frombuffer(response.content, dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    
    # HSL
    img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    h, l, s = cv2.split(img_hls)
    h_mean = np.mean(h)
    s_mean = np.mean(s)
    l_mean = np.mean(l)

    # HSV
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv) 
    h_mean = np.mean(h)
    s_mean = np.mean(s)
    v_mean = np.mean(v) 
    
    # Contrast
    l_min = np.min(l)
    l_max = np.max(l)
    l_normalized = (l - l_min) / (l_max - l_min)
    contrast = np.std(l_normalized)
    
    # Person detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    person_probability = int(len(faces) > 0)

    #Энтропия
    image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  
    
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist = hist.ravel() / hist.sum()
    hist = hist[hist != 0]

    entropy_value = -np.sum(hist * np.log2(hist))
    
    return [h_mean, v_mean, l_mean, contrast, person_probability, entropy_value]

urls = df['Thumb'].tolist()

results = []
for url in tqdm(urls, desc="Processing images"):
    try:
        metrics = analyze_image(url)
        results.append([url] + metrics)
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        results.append([url] + [None] * 5)  # Добавить пустые значения для неудачных изображений

result_df = pd.DataFrame(results, columns=['Thumb', 'H_mean','Brightness', 'L_mean', 'Contrast', 'Person_probability',
                                            'Entropy_value'])

Processing images: 100%|██████████████████████████| 9/9 [00:03<00:00,  2.70it/s]


## Цветовая гамма

In [4]:
#Цветовая гамма
result_df['Temp_cool'] = result_df['H_mean'].apply(lambda x: 1 if 90 <= x <= 270 else 0)
result = result_df[['Thumb', 'Brightness', 'L_mean', 'Contrast', 'Person_probability',
                    'Entropy_value','Temp_cool']]
result.drop_duplicates(inplace=True)
result

/var/folders/4l/h77mjd0979l8xgdkw8mttfjm0000gn/T/ipykernel_1231/2614486093.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop_duplicates(inplace=True)


,Thumb,Brightness,L_mean,Contrast,Person_probability,Entropy_value,Temp_cool
0,https://basket-10.wbbasket.ru/vol1495/part1495...,198.144383,181.693833,0.159844,0,7.054539,0
1,https://basket-17.wbbasket.ru/vol2665/part2665...,156.375582,145.214394,0.290165,0,7.665070,0
2,https://basket-17.wbbasket.ru/vol2665/part2665...,171.679395,160.900964,0.252590,0,7.528644,0
3,https://basket-12.wbbasket.ru/vol1790/part1790...,218.526324,215.070023,0.169335,0,4.289389,1
4,https://basket-13.wbbasket.ru/vol2013/part2013...,239.683869,225.270685,0.193085,0,2.577599,0
5,https://basket-17.wbbasket.ru/vol2713/part2713...,218.974829,218.438566,0.277011,0,2.231401,0
6,https://basket-14.wbbasket.ru/vol2178/part2178...,165.017214,157.180188,0.300323,0,6.831378,0
7,https://basket-14.wbbasket.ru/vol2122/part2122...,206.641111,190.986751,0.210521,0,6.702322,0
8,https://basket-14.wbbasket.ru/vol2074/part2074...,218.509518,209.015579,0.162709,0,6.705668,1


## Площадь и правило третей

In [33]:
#Обработка на класс объекта YOLO
import requests
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import warnings
from urllib3.exceptions import InsecureRequestWarning
from urllib.request import urlopen
import matplotlib.pyplot as plt
from ultralytics import YOLO
from scipy.fftpack import fft2, ifft2

# Загружаем все модели в список
models = {
    "yolov8n": YOLO('yolov8n.pt'),
    "yolov8l": YOLO('yolov8l.pt'),
    "yolov8m": YOLO('yolov8m.pt'),
    "yolov8x": YOLO('yolov8x.pt')
} 

def compute_normalized_offsets_to_thirds(image, keypoints):

    height, width = image.shape[:2]

    # Линии третей
    third_x1, third_x2 = width / 3, 2 * width / 3
    third_y1, third_y2 = height / 3, 2 * height / 3

    horizontal_offsets = []
    vertical_offsets = []

    if keypoints is None or len(keypoints) == 0: 
        return 0, 0

    for cx, cy in keypoints: 
        dist_x1 = cx - third_x1
        dist_x2 = cx - third_x2
        horizontal_offset = (dist_x1 if abs(dist_x1) < abs(dist_x2) else dist_x2) / width

        dist_y1 = cy - third_y1
        dist_y2 = cy - third_y2
        vertical_offset = (dist_y1 if abs(dist_y1) < abs(dist_y2) else dist_y2) / height

        horizontal_offsets.append(horizontal_offset)
        vertical_offsets.append(vertical_offset)

    avg_horizontal_offset = np.mean(horizontal_offsets)
    avg_vertical_offset = np.mean(vertical_offsets)

    return avg_horizontal_offset, avg_vertical_offset


def analyze_yolo(url, model):
    try:
        response = requests.get(url, stream=True, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
        return [], 0, 0  

    img_array = np.frombuffer(response.content, dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Could not decode image from {url}")
        return [], 0, 0

    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model(image_rgb, verbose=False)

    # Получение размеров изображения
    total_image_area = image_rgb.shape[0] * image_rgb.shape[1]

    area_ratios = []
    keypoints = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy().astype(int)  # Получаем координаты углов
        for x1, y1, x2, y2 in boxes:
            cx = (x1 + x2) / 2  # Вычисляем центр
            cy = (y1 + y2) / 2
            keypoints.append((cx, cy))

            box_area = (x2 - x1) * (y2 - y1)
            area_ratio = box_area / total_image_area
            area_ratios.append(area_ratio)
        
    avg_area = np.mean(area_ratios) if area_ratios else 0

    if keypoints:
        avg_keypoints = np.mean(keypoints, axis=0)
        rot_h, rot_v = compute_normalized_offsets_to_thirds(img, [avg_keypoints])
    else:
        rot_h, rot_v = 0, 0

    return avg_area, rot_h, rot_v


urls = df['Thumb'].tolist()
results = []

for url in tqdm(urls, desc="Processing images"):
    row = {"Thumb": url}
    for model_name, model in models.items():
        try:
            avg_area, rot_h, rot_v = analyze_yolo(url, model)
            row[f"area_{model_name}"] = avg_area  
            row[f"rot_h_{model_name}"] = rot_h
            row[f"rot_v_{model_name}"] = rot_v
        except Exception as e:
            print(f"An error occurred: {e}") 
            row[f"area_{model_name}"] = None
            row[f"rot_h_{model_name}"] = None
            row[f"rot_v_{model_name}"] = None

    results.append(row)

result_df_yolo = pd.DataFrame(results)
print(result_df_yolo)

Processing images: 100%|██████████████████████████| 9/9 [00:23<00:00,  2.67s/it]

                                               Thumb  area_yolov8n  \
0  https://basket-10.wbbasket.ru/vol1495/part1495...      0.225436   
1  https://basket-17.wbbasket.ru/vol2665/part2665...      0.580607   
2  https://basket-17.wbbasket.ru/vol2665/part2665...      0.531640   
3  https://basket-12.wbbasket.ru/vol1790/part1790...      0.357686   
4  https://basket-13.wbbasket.ru/vol2013/part2013...      0.000000   
5  https://basket-17.wbbasket.ru/vol2713/part2713...      0.000000   
6  https://basket-14.wbbasket.ru/vol2178/part2178...      0.684402   
7  https://basket-14.wbbasket.ru/vol2122/part2122...      0.896292   
8  https://basket-14.wbbasket.ru/vol2074/part2074...      0.376450   

   rot_h_yolov8n  rot_v_yolov8n  area_yolov8l  rot_h_yolov8l  rot_v_yolov8l  \
0      -0.095528       0.001016      0.411474       0.099593      -0.000508   
1      -0.166667       0.001016      0.566850       0.162602      -0.003557   
2      -0.152439       0.002541      0.528703      -0.160569  

In [35]:
import pandas as pd
import numpy as np

area_cols = [col for col in result_df_yolo.columns if col.startswith("area_")]
rot_h_cols = [col for col in result_df_yolo.columns if col.startswith("rot_h_")]
rot_v_cols = [col for col in result_df_yolo.columns if col.startswith("rot_v_")]

result_df_yolo["Area_ratio_yolov8x"] = result_df_yolo[area_cols].mean(axis=1, skipna=True)
result_df_yolo["rot_h"] = result_df_yolo[rot_h_cols].mean(axis=1, skipna=True)
result_df_yolo["rot_v"] = result_df_yolo[rot_v_cols].mean(axis=1, skipna=True)

result_df_end = result_df_yolo[["Thumb", "Area_ratio_yolov8x", "rot_h", "rot_v"]]

result_df_end

,Thumb,Area_ratio_yolov8x,rot_h,rot_v
0,https://basket-10.wbbasket.ru/vol1495/part1495...,0.363149,0.049797,2.540650e-04
1,https://basket-17.wbbasket.ru/vol2665/part2665...,0.573288,0.080285,-8.892276e-04
2,https://basket-17.wbbasket.ru/vol2665/part2665...,0.528492,-0.159045,1.016260e-03
3,https://basket-12.wbbasket.ru/vol1790/part1790...,0.270440,-0.122459,-3.810976e-04
4,https://basket-13.wbbasket.ru/vol2013/part2013...,0.062401,0.040650,2.540650e-04
5,https://basket-17.wbbasket.ru/vol2713/part2713...,0.201557,0.040650,2.168404e-17
6,https://basket-14.wbbasket.ru/vol2178/part2178...,0.574207,-0.097561,3.614075e-02
7,https://basket-14.wbbasket.ru/vol2122/part2122...,0.520691,-0.034553,7.012195e-02
8,https://basket-14.wbbasket.ru/vol2074/part2074...,0.094113,0.039126,-1.270325e-04


## Текст

In [37]:
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import easyocr
import cv2

def detect_text_with_easyocr(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image = np.array(Image.open(BytesIO(response.content)))

        reader = easyocr.Reader(['ru', 'en'])
        results = reader.readtext(image)

        all_text = " ".join([res[1] for res in results])
        has_text = int(len(all_text) > 0)
        text_regions = len(results)
        char_count_sum = len(all_text)
        return has_text, text_regions, char_count_sum

    except Exception as e:
        print(f"Ошибка: {e}")
        return None, None, None

urls = df['Thumb'].tolist()

results = []
for url in tqdm(urls, desc="Processing images"):
    try:
        metrics = detect_text_with_easyocr(url)
        results.append([url] + list(metrics))
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        results.append([url] + [None] * 3) 

result_df_text = pd.DataFrame(results, columns=['Thumb', 'has_text','text_regions', 'char_count_sum'])
result_df_text

Processing images: 100%|██████████████████████████| 9/9 [00:34<00:00,  3.81s/it]


,Thumb,has_text,text_regions,char_count_sum
0,https://basket-10.wbbasket.ru/vol1495/part1495...,1,11,64
1,https://basket-17.wbbasket.ru/vol2665/part2665...,1,9,75
2,https://basket-17.wbbasket.ru/vol2665/part2665...,1,9,74
3,https://basket-12.wbbasket.ru/vol1790/part1790...,1,10,150
4,https://basket-13.wbbasket.ru/vol2013/part2013...,1,2,39
5,https://basket-17.wbbasket.ru/vol2713/part2713...,0,0,0
6,https://basket-14.wbbasket.ru/vol2178/part2178...,0,0,0
7,https://basket-14.wbbasket.ru/vol2122/part2122...,1,5,39
8,https://basket-14.wbbasket.ru/vol2074/part2074...,1,4,37


## Соберем все вместе

In [38]:
df_merged = pd.merge(result_df_text, result_df_end, on='Thumb', how='outer')
df_merged = pd.merge(df_merged, result, on='Thumb', how='outer')
df_merged

,Thumb,has_text,text_regions,char_count_sum,Area_ratio_yolov8x,rot_h,rot_v,Brightness,L_mean,Contrast,Person_probability,Entropy_value,Temp_cool
0,https://basket-10.wbbasket.ru/vol1495/part1495...,1,11,64,0.363149,0.049797,2.540650e-04,198.144383,181.693833,0.159844,0,7.054539,0
1,https://basket-17.wbbasket.ru/vol2665/part2665...,1,9,75,0.573288,0.080285,-8.892276e-04,156.375582,145.214394,0.290165,0,7.665070,0
2,https://basket-17.wbbasket.ru/vol2665/part2665...,1,9,74,0.528492,-0.159045,1.016260e-03,171.679395,160.900964,0.252590,0,7.528644,0
3,https://basket-12.wbbasket.ru/vol1790/part1790...,1,10,150,0.270440,-0.122459,-3.810976e-04,218.526324,215.070023,0.169335,0,4.289389,1
4,https://basket-13.wbbasket.ru/vol2013/part2013...,1,2,39,0.062401,0.040650,2.540650e-04,239.683869,225.270685,0.193085,0,2.577599,0
5,https://basket-17.wbbasket.ru/vol2713/part2713...,0,0,0,0.201557,0.040650,2.168404e-17,218.974829,218.438566,0.277011,0,2.231401,0
6,https://basket-14.wbbasket.ru/vol2178/part2178...,0,0,0,0.574207,-0.097561,3.614075e-02,165.017214,157.180188,0.300323,0,6.831378,0
7,https://basket-14.wbbasket.ru/vol2122/part2122...,1,5,39,0.520691,-0.034553,7.012195e-02,206.641111,190.986751,0.210521,0,6.702322,0
8,https://basket-14.wbbasket.ru/vol2074/part2074...,1,4,37,0.094113,0.039126,-1.270325e-04,218.509518,209.015579,0.162709,0,6.705668,1


In [39]:
final_df = pd.merge(df, df_merged, on='Thumb', how='left')
final_df

,SKU,Item combination ID,Name,Category,Brand,Seller,Color,Balance,Comments,Final price,...,char_count_sum,Area_ratio_yolov8x,rot_h,rot_v,Brightness,L_mean,Contrast,Person_probability,Entropy_value,Temp_cool
0,149575719,126693294,Носки капроновые телесные набор 10 пар,Акции/Распродажа 11.11/Женщинам/Белье/Колготки...,NIKII,ИП Адюшкин Н Р,светло-бежевый,286,18759,136,...,64,0.363149,0.049797,2.540650e-04,198.144383,181.693833,0.159844,0,7.054539,0
1,266510489,244669663,Кроссовки зимние,NaN,Thomas Mitchell,ИП Нестеров И А,"красный, черный",17,26,5876,...,75,0.573288,0.080285,-8.892276e-04,156.375582,145.214394,0.290165,0,7.665070,0
2,266508481,245279769,Кроссовки зимние,NaN,Thomas Mitchell,ИП Нестеров И А,"зеленый, серый, бежевый, синий",75,24,6247,...,74,0.528492,-0.159045,1.016260e-03,171.679395,160.900964,0.252590,0,7.528644,0
3,179066374,163341676,кроссовки,NaN,Trendy Feet,Trendy Feet,бело-синий,21,98,3360,...,150,0.270440,-0.122459,-3.810976e-04,218.526324,215.070023,0.169335,0,4.289389,1
4,201392334,182598418,LITE Полуботинки женские осень натуральная кож...,Акции/Распродажа 11.11/Обувь/Женская/Ботинки и...,Mascotte,ООО Радость,"желтый, горчичный, желто-оранжевый",24,487,3179,...,39,0.062401,0.040650,2.540650e-04,239.683869,225.270685,0.193085,0,2.577599,0
5,271353738,155081746,Туфли классические кожаные броги,Акции/Распродажа 11.11/Обувь/Мужская/Ботинки и...,Mascotte,ООО Радость,"темно-коричневый, коричневый, темный горький ш...",70,62,7850,...,0,0.201557,0.040650,2.168404e-17,218.974829,218.438566,0.277011,0,2.231401,0
6,217831686,154017289,Кроссовки на шнурках,Акции/Распродажа 11.11/Обувь/Детская/Для мальч...,PlayToday,АО Плэйтудэй СНГ,"серый, зеленый, желтый",41,839,1276,...,0,0.574207,-0.097561,3.614075e-02,165.017214,157.180188,0.300323,0,6.831378,0
7,212290794,191730754,Лоферы женские замшевые,NaN,Ausgut,ИП Ходырев А Г,изумрудно-зеленый,48,91,4905,...,39,0.520691,-0.034553,7.012195e-02,206.641111,190.986751,0.210521,0,6.702322,0
8,207442697,187808846,Кроссовки Etonic Fanes,Акции/Распродажа 11.11/Спорт/Бег/Ходьба/Бег/Об...,Etonic,ООО Спортмастер,"розовый, фуксия",56,18,3336,...,37,0.094113,0.039126,-1.270325e-04,218.509518,209.015579,0.162709,0,6.705668,1


In [ ]:
# Сохраняем объединенные данные в новый файл CSV
output_path = '/Users/anastasia/Desktop/Научные страдания/Данные/Data_full.csv'
result.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

print("Объединение файлов CSV завершено. Результат сохранен в:", output_path)